<a href="https://colab.research.google.com/github/murmayasa/murmayasa/blob/main/Prediction_Of_Heart_Disease_Using_A_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

# Mengakses Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
csv_path = '/content/drive/MyDrive/Colab Notebooks/Heart_Disease_Prediction.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Membaca dataset

In [ ]:
data = pd.read_csv(csv_path)

# Memilih Kolom

In [ ]:
columns = ['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120', 'EKG results', 'Max HR',
           'Exercise angina', 'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease']
data = data[columns]

# Memisahkan fitur dan label

In [ ]:
X = data.drop('Heart Disease', axis=1)
y = data['Heart Disease']

# Normalisasi fitur menggunakan StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi dataset menjadi data latih dan data uji

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mengubah data menjadi bentuk yang sesuai untuk model RNN

In [ ]:
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

label_encoder = LabelEncoder()

# Ubah nilai dalam kolom target menjadi representasi numerik
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


# Pembuatan Model

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[1]), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Melatih model

In [ ]:
model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_rnn, y_test), callbacks=[early_stopping])

Epoch 1/50
7/7 [==============================] - 2s 99ms/step - loss: 0.6866 - accuracy: 0.5370 - val_loss: 0.6788 - val_accuracy: 0.5926
Epoch 2/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6650 - accuracy: 0.6759 - val_loss: 0.6610 - val_accuracy: 0.6667
Epoch 3/50
7/7 [==============================] - 0s 12ms/step - loss: 0.6448 - accuracy: 0.7407 - val_loss: 0.6431 - val_accuracy: 0.7222
Epoch 4/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6250 - accuracy: 0.7870 - val_loss: 0.6244 - val_accuracy: 0.7593
Epoch 5/50
7/7 [==============================] - 0s 8ms/step - loss: 0.6030 - accuracy: 0.7870 - val_loss: 0.6056 - val_accuracy: 0.7963
Epoch 6/50
7/7 [==============================] - 0s 8ms/step - loss: 0.5810 - accuracy: 0.8056 - val_loss: 0.5854 - val_accuracy: 0.8148
Epoch 7/50
7/7 [==============================] - 0s 9ms/step - loss: 0.5582 - accuracy: 0.8102 - val_loss: 0.5644 - val_accuracy: 0.8519
Epoch 8/50
7/7 [================

# Evaluasi model

In [ ]:
_, accuracy = model.evaluate(X_test_rnn, y_test)
print('Accuracy: %.2f%%' % (accuracy * 100))

2/2 [==============================] - 0s 8ms/step - loss: 0.3139 - accuracy: 0.8704
Accuracy: 87.04%
